In [ ]:
# train_model.py
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

CSV_PATH = "synthetic_wsht_weather_migraine_prob_600days_hourly_FIXED.csv"
FEATURES = ["workload_0_10", "stress_0_10", "hrv_rmssd_ms"]
TARGET_LABEL = "migraine_prob_next_hour"
SEQ_LEN = 24
RANDOM_SEED = 123

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

df = pd.read_csv(CSV_PATH, parse_dates=["timestamp"])
df = df.sort_values("timestamp").reset_index(drop=True)
df = df[~df[TARGET_LABEL].isna()].reset_index(drop=True)
df[TARGET_LABEL] = df[TARGET_LABEL].astype(int)

def build_sequences(frame, features, seq_len):
    scaler = StandardScaler()
    scaler.fit(frame[features])
    joblib.dump(scaler, "scaler.pkl")  # save scaler

    feat = scaler.transform(frame[features])
    y = frame[TARGET_LABEL].to_numpy().astype(int)
    X_list, y_list = [], []
    for end in range(seq_len, len(frame)):
        start = end - seq_len
        X_list.append(feat[start:end, :])
        y_list.append(y[end])
    return np.array(X_list), np.array(y_list)

X, y = build_sequences(df, FEATURES, SEQ_LEN)

# split chronologically
N = len(X)
train_n = int(N * 0.7)
val_n   = int(N * 0.15)
X_train, y_train = X[:train_n], y[:train_n]
X_val,   y_val   = X[train_n:train_n+val_n], y[train_n:train_n+val_n]
X_test,  y_test  = X[train_n+val_n:], y[train_n+val_n:]

# Diagnostic: Check class distribution
print("=" * 60)
print("CLASS DISTRIBUTION DIAGNOSTIC")
print("=" * 60)

# Count positive samples
train_pos = y_train.sum()
val_pos = y_val.sum()
test_pos = y_test.sum()

print(f"\nTraining set:")
print(f"  Total: {len(y_train)}, Positive (migraine=1): {train_pos} ({100*train_pos/len(y_train):.2f}%)")

print(f"\nValidation set:")
print(f"  Total: {len(y_val)}, Positive (migraine=1): {val_pos} ({100*val_pos/len(y_val):.2f}%)")

print(f"\nTest set:")
print(f"  Total: {len(y_test)}, Positive (migraine=1): {test_pos} ({100*test_pos/len(y_test):.2f}%)")

print("\n" + "=" * 60)
if val_pos > 10 and test_pos > 10:
    print("✓ GOOD: Enough positive cases to train effectively")
else:
    print("✗ WARNING: Not enough positive cases. Regenerate CSV with higher coefficients!")
print("=" * 60)

def make_model(input_len, input_dim):
    inputs = keras.Input(shape=(input_len, input_dim))
    x = layers.Masking(mask_value=0.0)(inputs)
    x = layers.LSTM(64)(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(32, activation="relu")(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss="binary_crossentropy",
        metrics=[keras.metrics.AUC(name="auc"), "accuracy"],
    )
    return model

model = make_model(SEQ_LEN, len(FEATURES))

callbacks = [
    keras.callbacks.ReduceLROnPlateau(monitor="val_auc", mode="max", factor=0.5, patience=3, verbose=1),
    keras.callbacks.EarlyStopping(monitor="val_auc", mode="max", patience=8, restore_best_weights=True, verbose=1),
]

history = model.fit(
    X_train, y_train,
    class_weight={0: 1.0, 1: 2.5},
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=64,
    callbacks=callbacks,
    verbose=1,
)

print("Test evaluation:", model.evaluate(X_test, y_test, verbose=0))
model.save("lstm_migraine_model.h5")
print("Saved lstm_migraine_model.h5 and scaler.pkl")


Epoch 1/50
158/158 [==============================] - 10s 33ms/step - loss: 0.0767 - auc: 0.0000e+00 - accuracy: 0.9856 - val_loss: 5.7681e-05 - val_auc: 0.0000e+00 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 2/50
158/158 [==============================] - 3s 20ms/step - loss: 6.5377e-05 - auc: 0.0000e+00 - accuracy: 1.0000 - val_loss: 2.2888e-05 - val_auc: 0.0000e+00 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/50
158/158 [==============================] - 3s 20ms/step - loss: 3.2573e-05 - auc: 0.0000e+00 - accuracy: 1.0000 - val_loss: 1.2172e-05 - val_auc: 0.0000e+00 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 4/50
157/158 [============================>.] - ETA: 0s - loss: 2.0005e-05 - auc: 0.0000e+00 - accuracy: 1.0000
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
158/158 [==============================] - 3s 21ms/step - loss: 1.9993e-05 - auc: 0.0000e+00 - accuracy: 1.0000 - val_loss: 7.6722e-06 - val_auc: 0.0000e+00 - val_accuracy: 1.0000 - lr: 0.0010
Epoc